# Explore AlphaEarth satellite embeddings

In this notebook you will see how to display and use AlphaEarth satellite embeddings.

Alpha Earth Embeddings are high-dimensional vector representations of locations on Earth, generated by a sophisticated AI model. These vectors encode the rich, multi-faceted context of a geographic point, allowing to understand and reason about places in a way that goes far beyond simple coordinates.

What information might an "Earth Embedding" capture?

- Physical Geography: Is it a mountain, a valley, a coastal region, a desert?
- Human Geography: Is it a dense urban center, a suburban area, rural farmland?
- Infrastructure: Proximity to roads, railways, power grids, bodies of water.
- Climate & Weather: Average temperature, rainfall, sunlight.
- Economic Activity: Land use, type of industry nearby.
- Ecological Context: Biome type, biodiversity.

So, a point in a forest and a point in a desert would have very different embedding vectors, even if their latitudes are similar.


References:

- AlphaEarth ref page: https://leafmap.org/maplibre/AlphaEarth/
- Dataset description: https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_SATELLITE_EMBEDDING_V1_ANNUAL#description

# Libraries

In [1]:
# original tutorial from https://leafmap.org/maplibre/AlphaEarth/
import ee
import leafmap.maplibregl as leafmap
import numpy as np
from IPython.display import display, HTML

c:\Users\degas\Documents\Github\YouTube\GeoSpatial\ee\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.11) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


# Initialization

In [2]:
ee.Authenticate()
# "force" option is to force the authentification and be sure to use the correct account
# if you have multiple accounts
# ee.Authenticate(force=True)

# to learn to create a project you can refer to:
# https://developers.google.com/earth-engine/guides/access
# here you should put your project code
ee.Initialize(project="alphaearth-476700")

In [3]:
# if you want to check the account you're using
# import glob, os
# for f in glob.glob(os.path.expanduser("~/.config/earthengine/credentials*")):
#    print(f)

# 3D visualization

Visualize embeddings in 3D.
You can select the year to apply the embeddings.

In [4]:
m = leafmap.Map(projection="globe", sidebar_visible=True)
m.add_basemap("USGS.Imagery")
m.add_alphaearth_gui()
m

Html(children=[<leafmap.maplibregl.Map object at 0x0000026A5EE2A350>, Card(children=[Row(children=[Btn(childre…

In [5]:
m = leafmap.Map(projection="globe", sidebar_visible=True)
m.add_basemap("USGS.Imagery")
m

Html(children=[<leafmap.maplibregl.Map object at 0x000001E1C7794D90>, Card(children=[Row(children=[Btn(childre…

### 2D view

You can select an AOI and display the map

In [5]:
lon = -121.8036
lat = 39.0372
m.set_center(lon, lat, zoom=12)

In [6]:
point = ee.Geometry.Point(lon, lat)
dataset = ee.ImageCollection("GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL")

# Change detection

In [7]:
# Each pixel in these Earth Engine images represents a vector of features — 64 high-dimensional
# embeddings that encode semantic information about land cover, crop type, or other spatial patterns learned by a model like AlphaEarth.

# So in practice in order to detect change between one year to the other we can do the dot product
# The dot product image highlights areas where the land cover changed semantically between the two dates.
# It’s a simple change detection proxy in embedding space — not raw reflectance difference, but feature-level difference.

In [9]:
# select 2022 embeddings and filter by the defined AOI
image1 = dataset.filterDate("2022-01-01", "2023-01-01").filterBounds(point).first()
# 2023
image2 = dataset.filterDate("2023-01-01", "2024-01-01").filterBounds(point).first()

In [10]:
# Check 1 image

# 1. Get the full metadata (client-side operation)
image_info = image1.getInfo()

# 2. Access the 'bands' list (usually the first band is enough)
# The dimensions are stored as a list [width, height]
if image_info and 'bands' in image_info and image_info['bands']:
    band_info = image_info['bands'][0]

    if 'dimensions' in band_info:
        width, height = band_info['dimensions']
        print(f"Image Width: {width} pixels")
        print(f"Image Height: {height} pixels")
    else:
        print("Dimensions not found in the band metadata.")
else:
    print("Image info or bands list is empty.")

Image Width: 16384 pixels
Image Height: 16384 pixels


In [11]:
vis_params = {"min": -0.3, "max": 0.3, "bands": ["A01", "A16", "A09"]}
m.add_ee_layer(image1, vis_params, name="Year 1 embeddings")
m.add_ee_layer(image2, vis_params, name="Year 2 embeddings")

In [23]:
m

Html(children=[<leafmap.maplibregl.Map object at 0x000001E1C7DB37F0>, Card(children=[Btn(children=[Icon(childr…

In [13]:
# Each pixel’s brightness encodes how similar the embeddings are between the two years:
# White (low value ~0): low similarity → large change (the model’s representation changed a lot)
# Black (high value ~1): high similarity → small change (land cover remained similar)

In [12]:
dot_prod = image1.multiply(image2).reduce(ee.Reducer.sum())

In [13]:
# let's see the image above
vis_params = {"min": 0, "max": 1, "palette": ["white", "black"]}
m.add_ee_layer(dot_prod, vis_params, name="Similarity")
# let's check out the previous opened leafmap window to observe the change detection 

# Studying over an area of interest

In [14]:
# Define your region of interest
roi = ee.Geometry.Rectangle([14, 48, 16, 50])  # [minLon, minLat, maxLon, maxLat]

# Load the embedding collection for 2023
embeddings_2023 = (
    ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL')
    .filterDate('2023-01-01', '2024-01-01')
    .filterBounds(roi)
)

In [15]:
# Mosaic tiles for that year and clip to ROI
img_2023 = embeddings_2023.mosaic().clip(roi)

# Print band names
print("Bands:", img_2023.bandNames().getInfo())

Bands: ['A00', 'A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30', 'A31', 'A32', 'A33', 'A34', 'A35', 'A36', 'A37', 'A38', 'A39', 'A40', 'A41', 'A42', 'A43', 'A44', 'A45', 'A46', 'A47', 'A48', 'A49', 'A50', 'A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A60', 'A61', 'A62', 'A63']


In [16]:
# Compute basic statistics (mean, std) over the region
stats = img_2023.reduceRegion(
    reducer=ee.Reducer.mean().combine(ee.Reducer.stdDev(), '', True),
    geometry=roi,
    scale=500,
    maxPixels=1e8
)
print("Stats for 2023 embeddings:", stats.getInfo())

Stats for 2023 embeddings: {'A00_mean': -0.09925572923119041, 'A00_stdDev': 0.08663869070958012, 'A01_mean': -0.1224455424633551, 'A01_stdDev': 0.06987952286370652, 'A02_mean': -0.09791871797465537, 'A02_stdDev': 0.06025968155340253, 'A03_mean': -0.08289566623150219, 'A03_stdDev': 0.06642226191436969, 'A04_mean': 0.1003268475874417, 'A04_stdDev': 0.05936514973594995, 'A05_mean': -0.10376792973166947, 'A05_stdDev': 0.07904517721226052, 'A06_mean': 0.13113930130801796, 'A06_stdDev': 0.07835463324940574, 'A07_mean': 0.08150295420904222, 'A07_stdDev': 0.11538706265977569, 'A08_mean': 0.08006836139489361, 'A08_stdDev': 0.05696298979353782, 'A09_mean': -0.01705041254314586, 'A09_stdDev': 0.05182579099779072, 'A10_mean': 0.03822635238704461, 'A10_stdDev': 0.08796197791215182, 'A11_mean': -0.03106918257005332, 'A11_stdDev': 0.060295964611672644, 'A12_mean': -0.11007137345313862, 'A12_stdDev': 0.042297909905279284, 'A13_mean': 0.0857067985829765, 'A13_stdDev': 0.06358765345646938, 'A14_mean': -

In [17]:
print("Stats for 2023 embeddings:", stats.getInfo())

# Visualization: pick 3 embedding bands for RGB
vis_params = {
    'bands': ['A01', 'A16', 'A09'],  # example bands for visualization
    'min': -0.3,
    'max': 0.3
}

# Create a map

# coordinates need to be passed as lon, lat
m = leafmap.Map(center=[14.5, 49.5], zoom=8)
m.add_ee_layer(img_2023, vis_params, 'Embeddings 2023')

Stats for 2023 embeddings: {'A00_mean': -0.09925572923119041, 'A00_stdDev': 0.08663869070958012, 'A01_mean': -0.1224455424633551, 'A01_stdDev': 0.06987952286370652, 'A02_mean': -0.09791871797465537, 'A02_stdDev': 0.06025968155340253, 'A03_mean': -0.08289566623150219, 'A03_stdDev': 0.06642226191436969, 'A04_mean': 0.1003268475874417, 'A04_stdDev': 0.05936514973594995, 'A05_mean': -0.10376792973166947, 'A05_stdDev': 0.07904517721226052, 'A06_mean': 0.13113930130801796, 'A06_stdDev': 0.07835463324940574, 'A07_mean': 0.08150295420904222, 'A07_stdDev': 0.11538706265977569, 'A08_mean': 0.08006836139489361, 'A08_stdDev': 0.05696298979353782, 'A09_mean': -0.01705041254314586, 'A09_stdDev': 0.05182579099779072, 'A10_mean': 0.03822635238704461, 'A10_stdDev': 0.08796197791215182, 'A11_mean': -0.03106918257005332, 'A11_stdDev': 0.060295964611672644, 'A12_mean': -0.11007137345313862, 'A12_stdDev': 0.042297909905279284, 'A13_mean': 0.0857067985829765, 'A13_stdDev': 0.06358765345646938, 'A14_mean': -

In [18]:
m

Html(children=[<leafmap.maplibregl.Map object at 0x0000026A5FE239A0>, Card(children=[Btn(children=[Icon(childr…

# K-means clustering

In [19]:
samples = img_2023.sample(
    region=roi,
    scale=10,
    numPixels=5000,
    geometries=True  # keep geometry to map clusters later
)

In [20]:
# Create the clusterer
k=5
clusterer = ee.Clusterer.wekaKMeans(nClusters=k)

# Train with number of clusters (k) passed in train() method
clusterer = clusterer.train(
    features=samples,
    inputProperties=img_2023.bandNames()
)

# Apply the clusterer to the whole image
result = img_2023.cluster(clusterer)

In [21]:
m = leafmap.Map(center=[14.5, 49.5], zoom=8)

# Cluster visualization
palette = ['#ff0000','#00ff00','#0000ff','#ffff00','#800080']  # length = k
k = len(palette)

vis_params = {
    'min': 0,
    'max': k-1,
    'palette': palette
}

# Visualize clusters
m.add_ee_layer(result.visualize(**vis_params), {}, "KMeans Clusters 2023")

# Layer control
m.add_layer_control()
m

# Define legend (ensure colors are valid HEX strings)
legend_labels = [f"Cluster {i}" for i in range(k)]
legend_colors = palette

# Display the map (in notebook this will render below)
display(m)

# --- Build an HTML legend you can always rely on ---
legend_items = "".join(
    f'''
    <div style="display:flex;align-items:center;margin:4px 0;">
      <div style="width:20px;height:14px;background:{color};margin-right:8px;border:1px solid #444;"></div>
      <div style="font-size:13px;color:#222;">Cluster {i}</div>
    </div>
    ''' for i, color in enumerate(palette)
)

legend_html = f"""
<div style="font-family: Arial, Helvetica, sans-serif;
            background: white;
            padding:10px;
            border-radius:6px;
            box-shadow: 0 1px 4px rgba(0,0,0,0.3);
            display:inline-block;">
  <div style="font-weight:600;margin-bottom:6px;">Cluster ID</div>
  {legend_items}
</div>
"""

# Show the legend below the map
display(HTML(legend_html))

Html(children=[<leafmap.maplibregl.Map object at 0x0000026A5FE68880>, Card(children=[Btn(children=[Icon(childr…

In [ ]:
# each cluster should correspond to a particular crop type
# we could for instance use a supervised learning algorithm to do pixelwise classification base don this embeddings
# starting from ground truths.